# Thymio Control System

This notebook contains the main control loop for the Thymio robot, integrating vision, state estimation, and motion control modules implemented in separate files.

In [ ]:
import numpy as np

from vision import VisionSystem
 
from control import ThymioController
from pathfinding import find_path
from utils import Pose, Point, euclidean_distance

from extended_kalman_filter import EKF 
from thymio_math_model import Thymio as tainy 


INITIAL_POSE = Pose(0, 0, 0)  # x, y, theta
INITIAL_COVARIANCE = np.eye(3) * 1e-1  # Initial uncertainty in state estimation
WAYPOINT_THRESHOLD = 3.0  # cm


def run():
    # A. Setup
    vis = VisionSystem()
    ekf = ExtendedKalmanFilter(initial_pose=INITIAL_POSE, initial_covariance=INITIAL_COVARIANCE)
    controller = ThymioController()
    
    # B. Map & Plan
    graph, start_node, goal_node = vis.construct_map(cspace_padding=2.0)
    waypoints_idx = find_path(graph, start_node, goal_node)
    waypoints: list[Point] = [graph.nodes[i]['pos'] for i in waypoints_idx]
    current_waypoint = 0
    
    # C. Loop
    while True:
        # 1. Pose Measurement & Sensing
        vision_pose_measurement = vis.get_robot_pose() # May be None
        prox_sensors = robot.get_prox()    # From Thymio API
        
        # 2. State Estimation via EKF
        ekf.predict(robot.left_speed, robot.right_speed, dt)
        if vision_pose_measurement:
            ekf.update(vision_pose_measurement)
        
        estimated_pose = ekf.get_state()
        
        # 3. Decision Making & Motion Control
        if euclidean_distance(Point(*estimated_pose[:2]), waypoints[current_waypoint]) < WAYPOINT_THRESHOLD:
            current_waypoint += 1  # Move to next waypoint

        motor_commands = controller.update(estimated_pose, waypoints[current_waypoint], prox_sensors)
        robot.set_motors(motor_commands)
    
